In [37]:
import regex
from collections import defaultdict

In [ ]:
corpus = [
    "Hello world!",
    "It is a great day to hug puppies.",
    "It would be a shame to not do so.",
    "This text is full of nonsense: I don't care!",
    "I hope I have enough pair variety here to get an interesting result.",
    "This project is going to be a challenge",
    "This is all about tokenization.",
    "I'm trying to make this tokenization algorithm work",
    "I really I hope this works.",
        ]

In [ ]:
def get_word_frequency(corpus):
    words = [word for text in corpus for word in text.split()]
    word_freq = {word:words.count(word) for word in words}
    return word_freq

def pretokenize_words(words):
    return {word:[c for c in word] for word in words}


def compute_pair_frequencies(tokens, word_freq):
    # Sets val to 0 (int()) for undefined key, as if all pairs were set to 0
    pair_freq = defaultdict(int)

    for word, freq in word_freq.items():
        token = tokens[word]
        if len(token) == 1:
            continue
        for i in range(len(token) - 1):
            pair = (token[i], token[i + 1])
            pair_freq[pair] += freq
    
    return pair_freq

def max_dict(dict, result):
    max_key = ''
    max_val = 0

    for key, val in dict.items():
        if val > max_val:
            max_key = key
            max_val = val

    if result == 'key':
        return max_key
    elif result == 'val':
        return max_val
    elif result == 'both':
        return max_key, max_val

In [60]:
# Define base alphabet, word frequencies and initial tokens of input words
word_freq = get_word_frequency(corpus)
tokens = pretokenize_words(word_freq.keys())
alphabet = list(set([c for word in word_freq.keys() for c in word]))

# Set target total vocab size and amount of merges to learn
target_vocab_size = 50
learned_vocab_size = target_vocab_size - len(alphabet)

# Set empty list of merges
merges = []

i=0
while i < learned_vocab_size:
    
    # Find most frequent pair, register the elements and the merge
    pair_freq = compute_pair_frequencies(tokens, word_freq)
    best_pair = max_dict(pair_freq, 'key') 
    merges.append(best_pair)
    alphabet.append(best_pair[0] + best_pair[1])

    for word in word_freq.keys():
        token = tokens[word]

        if len(token) == 1:
            continue

        # Sliding window over all pairs, checking if they match the elements
        # in the current best pair and replacing with a merge of the pair
        for j in range(len(token) - 1):
            if (token[j] == merges[i][0]) & (token [j+1] == merges[i][1]):
                new_token = token[:j] + [merges[i][0] + merges[i][1]] + token[j+2:]
                tokens[word] = new_token

    i += 1